<a href="https://colab.research.google.com/github/MateoUsme/Esp_Analitica/blob/main/04%20-%20%5BTALLER%5D%20-%20MAP%20REDUCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/UDEA-Esp-Analitica-y-Ciencia-de-Datos/EACD-03-BIGDATA/master/init.py
import init; init.init(force_download=False); 
from IPython.display import Image

replicating local resources


In [ ]:
Image("local/imgs/udea-datascience.png")

# **TALLER 1: MAP REDUCE**
## Desarrollado por: Mg. Efraín Alberto Oviedo
## Universidad de Antioquia

Por favor, ingrese la información de cada integrante del grupo de trabajo

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

#**Análisis de Accidentalidad en la ciudad de Medellín**

La secretaría de movilidad de la alcaldía de Medellín ha recolectado datos relacionados con la accidentalidad vial del año 2017 (Disponible en https://www.datos.gov.co) registrando en cada accidente la siguiente información: 
* Fecha
* Hora
* Día de la semana
* Clase de accidente
* Dirección
* Gravedad
* Barrio
* Comuna

Con el fin de adelantar programas que permitan disminuir la accidentalidad vial en la ciudad, se requiere de su ayuda para generar un informe que contenga la siguiente información:



In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz 
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

##**A) Porcentaje de accidentes en cada día de la semana**


In [5]:
data = sc.textFile("local/data/accidentalidad.csv").map(lambda x:x.split(","))
data_hdr = data.first()
data = data.filter(lambda x: x!=data_hdr)
data.take(5)

[['1565221',
  '01/01/2017',
  '00:10:00',
  'DOMINGO  ',
  'Atropello',
  'CL 68 CR 87',
  'EPM con Interior',
  'HERIDO',
  'Palenque',
  'Robledo',
  'Tramo de via'],
 ['1565189',
  '01/01/2017',
  '00:20:00',
  'DOMINGO  ',
  'Choque',
  'CL 44 CR 93',
  'EPM sin Interior',
  'HERIDO',
  'Campo Alegre',
  'La América',
  'Tramo de via'],
 ['1565182',
  '01/01/2017',
  '00:20:00',
  'DOMINGO  ',
  'Choque',
  'CR 16 CL 56',
  'Malla vial',
  'HERIDO',
  'Villatina',
  'Villa Hermosa',
  'Tramo de via'],
 ['1565404',
  '01/01/2017',
  '01:25:00',
  'DOMINGO  ',
  'Choque',
  'CR 41 CL 85',
  'Malla vial',
  'SOLO DAÑOS',
  'Las Granjas',
  'Manrique',
  'Interseccion'],
 ['1565185',
  '01/01/2017',
  '01:30:00',
  'DOMINGO  ',
  'Choque',
  'CR 76 CL 91',
  'Malla vial',
  'HERIDO',
  'Kennedy',
  'Doce de Octubre',
  'Tramo de via']]

In [6]:
data_hdr

['RADICADO',
 'FECHA',
 'HORA',
 'DIA',
 'CLASE',
 'DIRECCION',
 'TIPO_GEOCOD',
 'GRAVEDAD',
 'BARRIO',
 'COMUNA',
 'DISENO']

In [54]:
ans2 = data.map(lambda x: (x[3],1)).reduceByKey(lambda a,b: a+b)
totaaal = (ans2.values().sum())**-1
totalito = ans2.mapValues(lambda x: float(f"{x*totaaal*100:.1f}"))
totalito.takeOrdered(10,key=lambda x:-x[1])

[('VIERNES  ', 16.2),
 ('MARTES   ', 16.0),
 ('MIÉRCOLES', 15.5),
 ('JUEVES   ', 14.7),
 ('SÁBADO   ', 14.2),
 ('LUNES    ', 14.1),
 ('DOMINGO  ', 9.2)]

##**B) Barrio en el que se presenta el mayor y el menor número de accidentes**

In [8]:
ans1 = data.map(lambda x: (x[-3],1)).reduceByKey(lambda a,b: a+b)
print(f"Barrio con mayor nro: {ans1.max(lambda x:x[1])}\n\nBarrio con menor nro: {ans1.min(lambda x:x[1])}")

Barrio con mayor nro: ('La Candelaria', 944)

Barrio con menor nro: ('Villa Lilliam', 1)


##**C) Comuna en la que se presenta mayor número de accidentes con muertos**

In [9]:
ans3 = data.filter(lambda x: x[7].upper() == "MUERTO").map(lambda x: (x[-2],1)).reduceByKey(lambda a,b: a+b)
ans3.takeOrdered(3,key=lambda x:-x[1])

[('La Candelaria', 14), ('Guayabal', 11), ('Castilla', 10)]

##**D)	Día de la semana en que se presentaron mayor cantidad de accidentes con muertos**

In [26]:
ans4 = data.filter(lambda x: x[7].upper() == "MUERTO").map(lambda x: (x[3].replace(" ",""),1)).reduceByKey(lambda a,b: a+b)
ans4.takeOrdered(7,key=lambda x:-x[1])

[('DOMINGO', 31),
 ('SÁBADO', 28),
 ('LUNES', 24),
 ('MARTES', 22),
 ('JUEVES', 21),
 ('MIÉRCOLES', 12),
 ('VIERNES', 11)]

##**E)	Hora exacta y rango de hora en la que se presentó mayor cantidad de accidentes con heridos**

In [116]:
ans6 = data.filter(lambda x: x[7].upper() == "HERIDO").map(lambda x: (x[2],1)).reduceByKey(lambda a,b: a+b)
ans6.takeOrdered(10,key=lambda x:-x[1])

[('18:00:00', 447),
 ('19:00:00', 406),
 ('17:00:00', 401),
 ('16:00:00', 396),
 ('20:00:00', 384),
 ('07:00:00', 384),
 ('08:00:00', 374),
 ('15:00:00', 361),
 ('17:30:00', 356),
 ('14:00:00', 345)]

In [119]:
ans5 = data.filter(lambda x: x[7].upper() == "HERIDO").map(lambda x: (x[2].split(":")[0],1)).reduceByKey(lambda a,b: a+b)
ans5.takeOrdered(10,key=lambda x:-x[1])

[('07', 1537),
 ('17', 1532),
 ('18', 1515),
 ('06', 1485),
 ('12', 1370),
 ('16', 1331),
 ('19', 1310),
 ('13', 1273),
 ('14', 1234),
 ('15', 1233)]

#**Propuesta para disminuir la accidentalidad de la ciudad**

A la luz de los resultados obtenidos anteriormente, presente una propuesta orientada a mejorar las cifras de la accidentalidad en la ciudad.

##**1** Inicialmente partiremos de las comunas que acaparen por lo menos la mitad de los accidentes totales registrados

In [107]:
ans7 = data.map(lambda x: (x[-2],1)).reduceByKey(lambda a,b: a+b)
ans8 = data.filter(lambda x: (x[-2].startswith("La Can")) | (x[-2].startswith("Lau")) | (x[-2].startswith("Cast")) | (x[-2].startswith("El Po")))
ans9 = ans8.map(lambda x: (f"{x[-2]} - {x[7]}",1)).reduceByKey(lambda a,b: a+b)
sum_ans7 = (ans7.values().sum())**-1
sum_ans8 = (ans9.values().sum())**-1
total_1 = ans7.mapValues(lambda x: float(f"{x*sum_ans7*100:.0f}"))
total_2 = ans9.mapValues(lambda x: float(f"{x*sum_ans8*100:.0f}"))
total_1.takeOrdered(5,key=lambda x:-x[1])

[('La Candelaria', 20.0),
 ('Laureles Estadio', 12.0),
 ('Castilla', 10.0),
 ('El Poblado', 9.0),
 ('Guayabal', 8.0)]

Las primeras **cuatro comunas** en la lista contemplan ya más del **50%** de las comunas con alta tasa de accidentalidad en la ciudad, se concentra el análisis sobre estas comunas primordialmente


##**2** Entre las comunas con mayor accidentalidad, ¿de qué gravedad son los accidentes en porcentaje?

In [109]:
total_2.takeOrdered(10,key=lambda x:-x[1])

[('La Candelaria - SOLO DAÑOS', 22.0),
 ('La Candelaria - HERIDO', 17.0),
 ('Laureles Estadio - SOLO DAÑOS', 12.0),
 ('El Poblado - SOLO DAÑOS', 12.0),
 ('Castilla - HERIDO', 12.0),
 ('Laureles Estadio - HERIDO', 11.0),
 ('Castilla - SOLO DAÑOS', 8.0),
 ('El Poblado - HERIDO', 6.0),
 ('Castilla - MUERTO', 0.0),
 ('Laureles Estadio - MUERTO', 0.0)]

Si se analizan los datos de gravedad de accidente **podemos empezar a tener en cuenta tres cosas**:



*   Los accidentes fatales están por debajo del 1% entre los barrios con mayor accidentalidad
*   Los accidentes con consecuencia de solo daños materiales predominan en los barrios con mayor accidentalidad
*   La comuna *La Candelaria* es, entre accidentes con solo daños y heridos, la que más porcentaje de ellos tiene ameritando una intervención más drástica que en otras comunas

##**3** ¿Qué días de la semana tiene estas comunas la mayor cifra de accidentalidad?

In [113]:
ans10 = ans8.map(lambda x: (f"{x[-2]} - {x[3]}",1)).reduceByKey(lambda a,b: a+b)
total_3 = ans10.mapValues(lambda x: float(f"{x*sum_ans8*100:.0f}"))
total_3.takeOrdered(15,key=lambda x:-x[1])

[('La Candelaria - VIERNES  ', 7.0),
 ('La Candelaria - MARTES   ', 7.0),
 ('La Candelaria - MIÉRCOLES', 6.0),
 ('La Candelaria - JUEVES   ', 6.0),
 ('La Candelaria - LUNES    ', 5.0),
 ('La Candelaria - SÁBADO   ', 5.0),
 ('Laureles Estadio - MIÉRCOLES', 4.0),
 ('Laureles Estadio - MARTES   ', 4.0),
 ('Laureles Estadio - VIERNES  ', 4.0),
 ('Laureles Estadio - LUNES    ', 3.0),
 ('Castilla - MARTES   ', 3.0),
 ('El Poblado - MIÉRCOLES', 3.0),
 ('Castilla - MIÉRCOLES', 3.0),
 ('Laureles Estadio - JUEVES   ', 3.0),
 ('Castilla - SÁBADO   ', 3.0)]

La comuna *La Candelaria* tiene un registro de accidentes similares cada día de la semana entera, a diferencia de *Laureles Estadio* que dichos registros se sitúan entre los días comúnmente laborales al igual que *El Poblado* y *Castilla*.

##**Conclusiones**

Considerando los anteriores resultados generales y los enfocados en los barrios que representan la mitad de accidentes en toda la ciudad, se puede determinar el paso a seguir para atenuar el índice de accidentalidad:



*   Se encuentra a las comunas *La candelaria* y *Laureles Estadio* como las que más apoyo en cuanto a movilidad necesitan.
*   La comuna *La Candelaria* tiene un índice de accidentalidad similar en todos los días de la semana incluyendo sábado.
*   Principalmente los accidentes no son lo suficientemente graves para tener heridos pero sí son casi en su totalidad productores de daños materiales. Lo anterior puede verse reflejado en la imprudencia de las personas al frecuentar señales de PARE o semáforos y no acatarlas o por maniobras indebidas.
*   A lo largo de las horas pico se sitúan la mayor cantidad de accidentes, en la franja de 7:00 AM a 8:00 AM es donde más ocurren accidentes con heridos sin mucha diferencia a las franjas de 5:00 PM - 6:00 PM y 6:00 PM - 7:00 PM.
*   Aún así, revisando la hora exacta de accidentes, alrededor de las 6:00 PM es donde se tiene por gran diferencia el mayor registro de accidentes y esto debido a factores comunes como la congestión vehicular, el estrés, el cansancio al finalizar la jornada laboral, entre otros.

